Import packages and setting parameters

In [1]:
# packages
import numpy  as np 
import pandas as pd
import os
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --- logging --- #
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

# --- parameters --- #
# path tho the main table saved as csv
table_path = '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv'

# ------------------ #

logger.info(f'Taking the .csv file from {table_path}')

Taking the .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv


# Reading the `.csv` dataset

You need to download the table at  [Common MAGIC LST1 data - auto](https://docs.google.com/spreadsheets/d/1Tya0tlK-3fuN6_vXOU5FwJruis5vBA9FALAPLFHOfBQ/edit) and upload it

In [2]:
# column names of the table
vnames = ['Date', 'Source', 'Conditions', 'Trigger', 'Stereo', 'HV', 'MinZd [deg]', 'MaxZd [deg]',
          'LST runs', 'MAGIC first run', 'MAGIC last run', 'Obs time (min)', 'Same wobble (min)', '-']

# reading the table
logger.info(f'Reading .csv file from {table_path}')
df = pd.read_csv(table_path, names=vnames, index_col=False).drop('-', axis=1)
df = df[df['Source'].notna()][1:].reset_index(drop=True)

# now we convert magic runs to an array of runs
df['MAGIC runs'] = [np.arange(int(df['MAGIC first run'][n]), int(df['MAGIC last run'][n]) + 1, 1) for n in range(len(df))]    
df['LST runs']   = [np.fromstring(df['LST runs'][n][1:-1], dtype='int', sep=',')                  for n in range(len(df))] 
df.drop(['MAGIC first run', 'MAGIC last run'], axis=1, inplace=True)

# putting together LST and MAGIC run columns, interchanging places
df['Obs time (min)'], df.loc[:,'MAGIC runs'] = df['MAGIC runs'], df['Obs time (min)']
df.rename(columns={'Obs time (min)':'MAGIC runs','MAGIC runs':'Obs time (min)'}, inplace=True)

# updating some of the columns
for string in ['MinZd [deg]', 'MaxZd [deg]', 'Obs time (min)', 'Same wobble (min)']:
    df[string] = pd.to_numeric(df[string])

# adding a date column
df['Date'] = pd.to_datetime(df['Date'], format='%Y_%m_%d')   
    
display(df)

Reading .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv


Date        Source        Conditions          Trigger   Stereo  \
0   2019-11-23    CrabNebula          ['Dark']          ['Sum']   [True]   
1   2019-11-24    CrabNebula          ['Dark']          ['Sum']   [True]   
2   2019-11-26    CrabNebula          ['Dark']          ['L3T']   [True]   
3   2020-01-14    CrabNebula          ['Dark']         ['SumT']   [True]   
4   2020-01-15    CrabNebula  ['Dark', 'Moon']  ['SumT', 'L3T']   [True]   
..         ...           ...               ...              ...      ...   
399 2022-11-19  FRB20220912A          ['Dark']        ['L1_M2']  [False]   
400 2022-11-20  Boomerang-Ta          ['Dark']        ['L1_M2']  [False]   
401 2022-11-20    CrabNebula  ['Dark', 'Moon']        ['L1_M2']  [False]   
402 2022-11-20  FRB20220912A          ['Dark']        ['L1_M2']  [False]   
403 2022-11-20    GRB221009A  ['Moon', 'Dark']        ['L1_M2']  [False]   

              HV  MinZd [deg]  MaxZd [deg]  \
0    ['Nominal']         9.82        15.38   
1    ['Nominal']         8.26        14.49   
2    ['Nominal']         9.92        23.16   
3    ['Nominal']        17.39        29.16   
4    ['Nominal']        19.18        42.52   
..           ...          ...          ...   
399  ['Nominal']        27.73        36.97   
400  ['Nominal']        62.51        69.86   
401  ['Nominal']         6.97        47.73   
402  ['Nominal']        35.83        57.31   
403  ['Nominal']        44.88        70.21   

                                              LST runs  \
0                                               [1622]   
1                                   [1631, 1632, 1633]   
2                                   [1651, 1652, 1653]   
3                                         [1787, 1788]   
4                                   [1799, 1800, 1801]   
..                                                 ...   
399                                            [10868]   
400                              [10905, 10906, 10907]   
401  [10909, 10910, 10911, 10912, 10913, 10914, 109...   
402                                            [10904]   
403  [10892, 10893, 10894, 10895, 10896, 10897, 108...   

                                            MAGIC runs  Same wobble (min)  \
0                                   [5086812, 5086813]               0.00   
1                                   [5086858, 5086859]               0.00   
2                                   [5086952, 5086953]               0.00   
3                          [5088351, 5088352, 5088353]               0.00   
4    [5088408, 5088409, 5088410, 5088411, 5088412, ...               0.00   
..                                                 ...                ...   
399               [2006669, 2006670, 2006671, 2006672]              10.28   
400      [2006706, 2006707, 2006708, 2006709, 2006710]              32.94   
401  [2006713, 2006714, 2006715, 2006716, 2006717, ...             186.17   
402  [2006697, 2006698, 2006699, 2006700, 2006701, ...             126.71   
403  [2006680, 2006681, 2006682, 2006683, 2006684, ...             110.03   

     Obs time (min)  
0             29.66  
1             31.28  
2             45.12  
3             58.14  
4             77.78  
..              ...  
399           63.06  
400           61.91  
401          214.66  
402          131.99  
403          112.30  

[404 rows x 12 columns]

# Filtering conditions
For the different sources we want to study, in our case:
- **Crab Nebula**
- **BL Lac**
- **Mrk421**

In [4]:
# -- parameters -- #
source_names        = ['CrabNebula', 'BLLac', 'Mrk421']
source_common_names = ['Crab', 'BLLac', 'Mrk421']
min_same_wobble = 0
max_date        = '2022-07-01'

# filtering the dataset, the source, the wobble and the date
dfs = []
for source in source_names:
    df_tmp = df[(df['Source'] == source) & (df['Same wobble (min)'] >= min_same_wobble) & (df['Date'] < max_date)]
    logger.info(f'Filtering dataset with:\n--> Source {source}\n--> Minimum same wobble per night {min_same_wobble}\n--> Maximum date {max_date}\n\n-->Number of nights: {len(df_tmp)}')
    df_tmp = df_tmp.reset_index(drop=True)
    dfs.append(df_tmp)
    
    display(df_tmp.head(5))

Filtering dataset with:
--> Source CrabNebula
--> Minimum same wobble per night 0
--> Maximum date 2022-07-01

-->Number of nights: 40


Date      Source        Conditions          Trigger  Stereo  \
0 2019-11-23  CrabNebula          ['Dark']          ['Sum']  [True]   
1 2019-11-24  CrabNebula          ['Dark']          ['Sum']  [True]   
2 2019-11-26  CrabNebula          ['Dark']          ['L3T']  [True]   
3 2020-01-14  CrabNebula          ['Dark']         ['SumT']  [True]   
4 2020-01-15  CrabNebula  ['Dark', 'Moon']  ['SumT', 'L3T']  [True]   

            HV  MinZd [deg]  MaxZd [deg]            LST runs  \
0  ['Nominal']         9.82        15.38              [1622]   
1  ['Nominal']         8.26        14.49  [1631, 1632, 1633]   
2  ['Nominal']         9.92        23.16  [1651, 1652, 1653]   
3  ['Nominal']        17.39        29.16        [1787, 1788]   
4  ['Nominal']        19.18        42.52  [1799, 1800, 1801]   

                                          MAGIC runs  Same wobble (min)  \
0                                 [5086812, 5086813]                0.0   
1                                 [5086858, 5086859]                0.0   
2                                 [5086952, 5086953]                0.0   
3                        [5088351, 5088352, 5088353]                0.0   
4  [5088408, 5088409, 5088410, 5088411, 5088412, ...                0.0   

   Obs time (min)  
0           29.66  
1           31.28  
2           45.12  
3           58.14  
4           77.78

Filtering dataset with:
--> Source BLLac
--> Minimum same wobble per night 0
--> Maximum date 2022-07-01

-->Number of nights: 15


Date Source Conditions  Trigger  Stereo           HV  MinZd [deg]  \
0 2021-07-09  BLLac   ['Dark']  ['L3T']  [True]  ['Nominal']        21.60   
1 2021-07-10  BLLac   ['Dark']  ['L3T']  [True]  ['Nominal']        34.85   
2 2021-07-12  BLLac   ['Dark']  ['L3T']  [True]  ['Nominal']        40.50   
3 2021-07-16  BLLac   ['Dark']  ['L3T']  [True]  ['Nominal']        17.05   
4 2021-07-17  BLLac   ['Dark']  ['L3T']  [True]  ['Nominal']        15.40   

   MaxZd [deg]            LST runs          MAGIC runs  Same wobble (min)  \
0        27.04        [5219, 5220]  [5098298, 5098299]              25.02   
1        40.28        [5229, 5230]  [5098326, 5098327]              19.41   
2        46.76        [5251, 5252]  [5098367, 5098368]              16.52   
3        18.69              [5283]           [5098498]              12.00   
4        19.37  [5302, 5303, 5304]  [5098541, 5098542]              23.72   

   Obs time (min)  
0           26.15  
1           21.20  
2           30.21  
3           12.10  
4           32.09

Filtering dataset with:
--> Source Mrk421
--> Minimum same wobble per night 0
--> Maximum date 2022-07-01

-->Number of nights: 32


Date  Source Conditions  Trigger  Stereo                      HV  \
0 2020-01-31  Mrk421   ['Dark']  ['L3T']  [True]             ['Nominal']   
1 2020-06-18  Mrk421   ['Dark']  ['L3T']  [True]             ['Nominal']   
2 2020-12-12  Mrk421   ['Dark']  ['L3T']  [True]             ['Nominal']   
3 2020-12-14  Mrk421   ['Dark']  ['L3T']  [True]             ['Nominal']   
4 2021-02-12  Mrk421   ['Dark']  ['L3T']  [True]  ['Reduced', 'Nominal']   

   MinZd [deg]  MaxZd [deg]  \
0         9.51        24.59   
1        59.97        72.44   
2        10.12        52.27   
3        16.33        24.30   
4        17.67        30.44   

                                            LST runs  \
0                                       [1919, 1921]   
1                           [2114, 2115, 2116, 2117]   
2  [3216, 3217, 3218, 3219, 3220, 3221, 3222, 322...   
3                                       [3247, 3248]   
4                                 [3617, 3618, 3619]   

                                          MAGIC runs  Same wobble (min)  \
0               [5088933, 5088934, 5088935, 5088936]               0.00   
1                        [5089907, 5089908, 5089909]               0.00   
2  [5093614, 5093615, 5093616, 5093617, 5093618, ...             184.23   
3                        [5093691, 5093692, 5093693]              37.26   
4                                 [5094461, 5094462]               0.00   

   Obs time (min)  
0           60.62  
1           59.11  
2          191.21  
3           38.96  
4           58.35

# Generate MAGIC runs `.txt` to launch the slurm job calib $\rightarrow$ dl1

In [5]:
magic_paths  = [f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runs{source}.txt' for source in source_common_names]

for dframe, path in zip(dfs, magic_paths):
    # generating the text files for MAGIC runs
    # adding an upper label
    stringM = '# MAGIC runs (separed by commas for each night)'
    
    # adding for each night
    for n in range(len(dframe)):
        str_n   = ','.join(dframe['MAGIC runs'][n].astype(str))
        stringM = f'{stringM}\n# Night {n}\n{str_n}'

    with open(path, 'w') as f:
        f.write(stringM)

    # print(stringM)
    logger.info(f'File written: {path}\n')

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsBLLac.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsMrk421.txt



# Generate LST-MAGIC runs `.txt` to launch the slurm job dl1 $\rightarrow$ stereo
Will be the same `.txt` needed to launch the stereo $\rightarrow$ dl2

In [6]:
common_paths  = [f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_data{source}.txt' for source in source_common_names]

for dframe, path in zip(dfs, common_paths):
    
    # generating the text files for LST+MAGIC for each LST subrun repeating row
    # creating a label
    string = '#LST runs - MAGIC runs (separed by commas)'
    
    for n in range(len(dframe['LST runs'])):
        for s in range(len(dframe['LST runs'][n])):
            LSTsub, MRuns = dframe['LST runs'][n][s], ','.join(dframe['MAGIC runs'][n].astype(str))
            str_n  = f'{LSTsub}-{MRuns}'
            if s == 0:
                string = f'{string}\n# Night {n}\n{str_n}'
            else:
                string = f'{string}\n{str_n}'

    with open(path, 'w') as f:
        f.write(string)
        
    # print(string)
    logger.info(f'File written: {path}\n')

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataBLLac.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataMrk421.txt



# Statistics
We can extract some statistics from the common data `.csv` file

In [4]:
aa = df.loc[:232,].groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)
aa['Obs time (hour)'] = aa['Same wobble (min)']/60
display(aa)

/tmp/ipykernel_15162/4068288412.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aa = df.loc[:232,].groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)


MinZd [deg]  MaxZd [deg]  Same wobble (min)  Obs time (min)  \
Source                                                                      
BLLac              304.19       369.16             237.65          366.59   
1ES0647+250         32.83        98.44             289.02          297.63   
1ES1959+650        287.45       324.51             289.90          515.32   
Mrk421             426.10       643.61             592.28         1030.41   
M87                222.26       332.76             593.72          736.31   
RSOph              266.18       368.66             616.42          656.65   
GalacticCent      1347.45      1423.54             844.06         1084.89   
CrabNebula        1009.00      1655.85            1006.55         2574.82   
Mrk501             652.80       980.27            1295.70         1613.05   
LHAASOJ2108+       623.31       897.59            1660.42         1936.12   

              Obs time (hour)  
Source                         
BLLac                3.960833  
1ES0647+250          4.817000  
1ES1959+650          4.831667  
Mrk421               9.871333  
M87                  9.895333  
RSOph               10.273667  
GalacticCent        14.067667  
CrabNebula          16.775833  
Mrk501              21.595000  
LHAASOJ2108+        27.673667

In [15]:
aa = df.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)
aa['Same wobble (hour)'] = aa['Same wobble (min)']/60
display(aa)

aa1 = aa['Same wobble (hour)'].to_numpy()

/tmp/ipykernel_15162/1793899866.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aa = df.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)


MinZd [deg]  MaxZd [deg]  Same wobble (min)  Obs time (min)  \
Source                                                                      
RSOph              266.18       368.66             616.42          656.65   
1ES1959+650        644.31       727.46             730.22         1000.32   
GRB221009A         542.72       772.92             798.27          930.49   
M87                390.69       550.37             874.74         1074.70   
Mrk421             915.91      1336.16            1380.02         1946.63   
GalacticCent      2160.73      2278.77            1426.44         1821.65   
LHAASOJ2108+       623.31       897.59            1660.42         1936.12   
Mrk501             845.73      1248.70            1669.96         2056.65   
BLLac              911.19      1353.72            1880.31         2289.10   
CrabNebula        1225.09      2358.18            3019.10         5034.09   

              Same wobble (hour)  
Source                            
RSOph                  10.273667  
1ES1959+650            12.170333  
GRB221009A             13.304500  
M87                    14.579000  
Mrk421                 23.000333  
GalacticCent           23.774000  
LHAASOJ2108+           27.673667  
Mrk501                 27.832667  
BLLac                  31.338500  
CrabNebula             50.318333

In [27]:
s = 'M87'

aa = df.iloc[:232,].query(f'Source == "{s}"')

aa = aa.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)
aa['Same wobble (hour)'] = aa['Same wobble (min)']/60
display(aa)

aa2 = aa['Same wobble (hour)'].to_numpy()

aa = df.query(f'Source == "{s}"')

aa = aa.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)
aa['Same wobble (hour)'] = aa['Same wobble (min)']/60
display(aa)

aa1 = aa['Same wobble (hour)'].to_numpy()

aa2/aa1*100

/tmp/ipykernel_15162/1469540621.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aa = aa.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)


MinZd [deg]  MaxZd [deg]  Same wobble (min)  Obs time (min)  \
Source                                                                
M87          222.26       332.76             593.72          736.31   

        Same wobble (hour)  
Source                      
M87               9.895333

/tmp/ipykernel_15162/1469540621.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aa = aa.groupby(['Source']).sum().sort_values('Same wobble (min)').tail(10)


MinZd [deg]  MaxZd [deg]  Same wobble (min)  Obs time (min)  \
Source                                                                
M87          390.69       550.37             874.74          1074.7   

        Same wobble (hour)  
Source                      
M87                 14.579

array([67.87388253])